In [ ]:
import pandas as pd

suicide_data = pd.read_csv("/content/Suicide_Detection.csv")
suicide_data.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [ ]:
suicide_data = suicide_data.drop(['Unnamed: 0'], axis=1)

suicide_data.head()

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [ ]:
import re

def clean_data(text):
  text = text.lower()  # Convert to lowercase
  text = re.sub(r'\W', ' ', text)  # Remove special characters
  text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
  return text

suicide_data['text'] = suicide_data['text'].apply(clean_data)

suicide_data.head()

In [ ]:
suicide_data.to_csv('cleaned_suicide_data.csv', index=False)

In [ ]:
suicide_data['text'][3]

'i need helpjust help me im crying so hard'

**Model With Cleaned_Dataset**


In [ ]:
!pip install -qqq transformers datasets

In [ ]:
import torch
import torch.nn as nn
import wandb
from sklearn.model_selection import train_test_split
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict

In [ ]:
import pandas as pd

suicide_data = pd.read_csv("/content/cleaned_suicide_data.csv")
suicide_data.head()

,text,class
0,ex wife threatening suiciderecently i left my ...,suicide
1,am i weird i don t get affected by compliments...,non-suicide
2,finally 2020 is almost over so i can never hea...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,i m so losthello my name is adam 16 and i ve b...,suicide


In [ ]:
suicide_data.shape

(232074, 2)

In [ ]:
suicide_data.isnull().sum()

,0
text,0
class,0


In [ ]:
suicide_data.dropna(inplace=True)

In [ ]:
suicide_data['class'].value_counts()

,count
class,
suicide,116037
non-suicide,116037


In [ ]:
suicide_data['label'] = suicide_data['class'].map({'suicide': 1, 'non-suicide': 0})
suicide_data.head()

,text,class,label
0,ex wife threatening suiciderecently i left my ...,suicide,1
1,am i weird i don t get affected by compliments...,non-suicide,0
2,finally 2020 is almost over so i can never hea...,non-suicide,0
3,i need helpjust help me im crying so hard,suicide,1
4,i m so losthello my name is adam 16 and i ve b...,suicide,1


In [ ]:
suicide_data = suicide_data.drop(['class'], axis=1)
suicide_data.head()

,text,label
0,ex wife threatening suiciderecently i left my ...,1
1,am i weird i don t get affected by compliments...,0
2,finally 2020 is almost over so i can never hea...,0
3,i need helpjust help me im crying so hard,1
4,i m so losthello my name is adam 16 and i ve b...,1


In [ ]:
suicide_train_data, suicide_test_data = train_test_split(suicide_data, test_size=0.2, random_state=42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def dataset_conversion(train, test):

  train_dataset = Dataset.from_pandas(train)
  test_dataset = Dataset.from_pandas(test)

  return DatasetDict({"train": train_dataset,
                      "test": test_dataset})

raw_datasets = dataset_conversion(suicide_train_data, suicide_test_data)

In [ ]:
def tokenize_function(dataset):
  return tokenizer(dataset["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/185645 [00:00<?, ? examples/s]

Map:   0%|          | 0/46412 [00:00<?, ? examples/s]

In [ ]:
crisis_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(30000))
test_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(5000))

In [ ]:
wandb.login()
wandb.init(project="Suicide_Detection")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gprashanth7301 (gprashanth7301-university-at-albany) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
crisis_model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
training_args = TrainingArguments(
    output_dir="suicide_detection",
    overwrite_output_dir = True,
    report_to = 'wandb',
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="steps",
    save_steps=2000,
    fp16 = True
)

trainer = Trainer(
    model=crisis_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

<ipython-input-29-24b168582f91>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
500,0.184400
1000,0.146900
1500,0.137700


TrainOutput(global_step=1875, training_loss=0.151144287109375, metrics={'train_runtime': 780.4926, 'train_samples_per_second': 38.437, 'train_steps_per_second': 2.402, 'total_flos': 7893331660800000.0, 'train_loss': 0.151144287109375, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.12085039913654327,
 'eval_model_preparation_time': 0.0153,
 'eval_runtime': 45.0396,
 'eval_samples_per_second': 111.013,
 'eval_steps_per_second': 6.949,
 'epoch': 1.0}

In [ ]:
wandb.finish()

eval/loss,▁
eval/model_preparation_time,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
test/loss,▁
test/model_preparation_time,▁
test/runtime,▁
test/samples_per_second,▁
test/steps_per_second,▁
train/epoch,▁▄▆██


In [ ]:
trainer.save_model("/content/crisis_detection_model")

In [ ]:

def predict_suicide(text):
    # Load the saved model and tokenizer
    model_path = "/content/crisis_detection_model"
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Preprocess the input text
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)

    # Make the prediction
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)

    # Determine the prediction label
    if predictions.item() == 1:
        return "Suicidal Message. Help Needed"
    else:
        return "Non-Suicidal Message. Safe Message"


#predict_suicide("I am so alone and sad")

In [ ]:
text_messages = ["I feel like I have no reason to live anymore.",
"I’ve been feeling stressed lately, but I’m trying to manage it.",
"I’m going through a tough time, but I know things will get better.",
"Work has been overwhelming, I just need a break.",
"I had a rough day, but I’m trying to stay positive.",
"I’m feeling anxious about my exams, but I hope to do well.",
"Sometimes I feel down, but talking to friends helps a lot.",
"Life is hard sometimes, but I know I have support.",
"Everything is falling apart, I just want this pain to end.",
"No one would care if I was gone. It’s better this way.",
"I’ve been thinking a lot about ending things. Nothing makes sense anymore.",
"I just need to vent about my problems, but I know I’ll be okay.",
"I don’t see a way out of this. I think I should just disappear.",
"I'm tired of pretending to be okay. I can't take this anymore.",
"I have been looking for ways to end my life. I just need peace.",
"Goodbye, I hope everyone finds happiness without me."
]

for message in text_messages:
  prediction = predict_suicide(message)
  print(message+ ":"+ prediction)

I feel like I have no reason to live anymore.:Suicidal Message. Help Needed
I’ve been feeling stressed lately, but I’m trying to manage it.:Non-Suicidal Message. Safe Message
I’m going through a tough time, but I know things will get better.:Non-Suicidal Message. Safe Message
Work has been overwhelming, I just need a break.:Non-Suicidal Message. Safe Message
I had a rough day, but I’m trying to stay positive.:Non-Suicidal Message. Safe Message
I’m feeling anxious about my exams, but I hope to do well.:Non-Suicidal Message. Safe Message
Sometimes I feel down, but talking to friends helps a lot.:Non-Suicidal Message. Safe Message
Life is hard sometimes, but I know I have support.:Non-Suicidal Message. Safe Message
Everything is falling apart, I just want this pain to end.:Suicidal Message. Help Needed
No one would care if I was gone. It’s better this way.:Suicidal Message. Help Needed
I’ve been thinking a lot about ending things. Nothing makes sense anymore.:Suicidal Message. Help Needed